In [6]:
# Import libraries 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

ModuleNotFoundError: No module named 'langchain'

In [2]:
PINECONE_API_KEY = '586e0dff-89bc-4bad-af43-6d96ea1b477c'
PINECONE_API_ENV = 'gcp-starter'

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf('data/')

In [12]:
# Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)
print('length of my text chucks:', len(text_chunks))

length of my text chucks: 6983


In [14]:
# Download embedding model 
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

.gitattributes: 100%|███████████████████████| 1.18k/1.18k [00:00<00:00, 410kB/s]
1_Pooling/config.json: 100%|███████████████████| 190/190 [00:00<00:00, 68.3kB/s]
README.md: 100%|███████████████████████████| 10.6k/10.6k [00:00<00:00, 2.87MB/s]
config_sentence_transformers.json: 100%|███████| 116/116 [00:00<00:00, 45.0kB/s]
data_config.json: 100%|█████████████████████| 39.3k/39.3k [00:00<00:00, 290kB/s]
pytorch_model.bin: 100%|███████████████████| 90.9M/90.9M [00:03<00:00, 24.2MB/s]
sentence_bert_config.json: 100%|█████████████| 53.0/53.0 [00:00<00:00, 23.3kB/s]
special_tokens_map.json: 100%|█████████████████| 112/112 [00:00<00:00, 42.5kB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 1.34MB/s]
tokenizer_config.json: 100%|████████████████████| 350/350 [00:00<00:00, 125kB/s]
train_script.py: 100%|█████████████████████| 13.2k/13.2k [00:00<00:00, 3.64MB/s]
vocab.txt: 100%|██████████████████████████████| 232k/232k [00:00<00:00, 597kB/s]
modules.json: 100%|█████████

In [16]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [17]:
query_result = embeddings.embed_query('Hello World')
print("Length", len(query_result))

Length 384


In [18]:
# Initializing Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
             environment=PINECONE_API_ENV)

index_name='medical-chatbot'

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], 
                                embeddings, 
                                index_name=index_name)

In [19]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = 'What are Allergies'

docs= docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.', metadata={}), Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxificationKEY TERMS\nAllergen —A foreign substance, such as mites in\nhouse dust or animal dander, that when\ninhaled,causes the airways to narrow and pro-\nduces symptoms of asthma.\nAntibody —A protein, also called immunoglobu-\nlin, produced by immune system cells to remove\nantigens (the foreign substances that trigger the\nimmune response).\nFibromyalgia 

In [20]:
prompt_template = '''
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
'''

In [24]:
PROMPT = PromptTemplate(template=prompt_template,
                       input_variables=['context', 'question'])
chain_type_kwargs={'prompt': PROMPT}

In [25]:
llm = CTransformers(model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                   model_type='llama',
                   config={'max_new_tokens': 512,
                          'temperature': 0.8})

In [26]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [27]:
while True:
    user_input=input(f'Input Prompt:')
    result=qa({'query': user_input})
    print("Response:", result['result'])

Input Prompt: what causes anxiety


Response: The cause of anxiety can vary depending on the individual, but some common causes include stress, trauma, genetics, brain chemistry imbalances, and certain medical conditions. It's important to consult a mental health professional for proper diagnosis and treatment if you are experiencing persistent or severe anxiety.


KeyboardInterrupt: Interrupted by user